In [1]:
try:
    import sys
    import os
    sys.path.insert(0, os.path.abspath(os.curdir))
except ModuleNotFoundError:
    pass
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium
import carregar_mapa_desagrupados as cmd
import carregar_dados_agrupados as cda


<p>1 - Iniciar sessão spark</p>

In [2]:
spark_session = cda.iniciar_sessao_spark()

23/09/30 17:40:46 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/30 17:40:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/30 17:40:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataframe_desagrupados = cmd.load_dataframe_desagrupados(spark=spark_session)
dataframe_desagrupados.show()

+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|            CONSOCIO|             EMPRESA|DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|           LATITUDE|          LONGITUDE|DATA_EXTRACAO_API|TURNO|
+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   2023-09-15|   08:50|          3054-10|               2|                     33743|         47399|         2023-09-15T11:50:12Z|         -23.598638|         -46.416157|       2023-09-15|Manhã|


In [4]:
df_prefixo_onibus = cmd.obter_prefixo_onibus(dataframes_desagrupados_completo=dataframe_desagrupados)
df_prefixo_onibus

[66580,
 15173,
 73951,
 78598,
 26486,
 61679,
 36102,
 31944,
 66070,
 11567,
 48280,
 22129,
 31703,
 15057,
 26543,
 77605,
 36103,
 45298,
 55474,
 66114,
 63212,
 31156,
 64442,
 71655,
 32098,
 68463,
 56242,
 52743,
 72935,
 52001,
 10422,
 61749,
 71530,
 48899,
 51418,
 21965,
 63676,
 52910,
 51428,
 64317,
 31762,
 11745,
 51056,
 72485,
 15194,
 45166,
 72926,
 45410,
 25207,
 16530,
 55426,
 45726,
 71429,
 16597,
 85760,
 52051,
 71285,
 71292,
 47745,
 71386,
 81572,
 36077,
 47050,
 71549,
 16673,
 81443,
 48963,
 52904,
 76085,
 77441,
 71548,
 45390,
 26140,
 85641,
 15221,
 68166,
 68047,
 63181,
 63448,
 86153,
 66911,
 73916,
 71255,
 51105,
 68752,
 76050,
 11771,
 47741,
 48203,
 63678,
 45724,
 63586,
 81038,
 55329,
 55093,
 77434,
 11860,
 45558,
 68314,
 51112,
 72928,
 31159,
 32204,
 47683,
 15133,
 11988,
 31111,
 47636,
 63109,
 47229,
 78203,
 52745,
 22808,
 31126,
 68608,
 45134,
 31102,
 80978,
 31832,
 36577,
 45473,
 16499,
 22707,
 45580,
 45474,


In [5]:
prefixo_onibus = '68712'

In [6]:
dataframe_filter = cmd.dataframe_filter_desagrupados(
    dataframes_desagrupados_completo=dataframe_desagrupados,
    data_extracao='2023-09-18',
    prefixo_onibus=prefixo_onibus,
    turno='Noite'
)
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         68712|                     33040|          546T-10|        -23.6541035|-46.710938999999996|
|   18:05|         68712|                     33040|          546T-10|       -23.65393525|-46.707456500000006|
|   18:10|         68712|                     33040|          546T-10|        -23.6543535|      -46.701634625|
|   18:15|         68712|                     33040|          546T-10|       -23.65908975|        -46.7012395|
|   18:20|         68712|                     33040|          546T-10|        -23.6613215|         -46.693996|
|   18:25|         68712|                     33040|          546T-10|         -23.670195|         -46.688707|
|

In [7]:
letreiro_onibus = cmd.obter_letreiro_onibus(dataframe_prefixo_onibus=dataframe_filter)
letreiro_onibus

'546T-10'

In [8]:
rota_linha = cmd.obter_rota_linha(letreiro_onibus=letreiro_onibus)
rota_linha[0]

['546T-10', 'Vl. Guacuri - Term. Sto. Amaro', '0082BA']

In [9]:
dataframe_filter = dataframe_filter.withColumn("DISTANCIA", cmd.haversine_udf(dataframe_filter["LATITUDE"], dataframe_filter["LONGITUDE"]))
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+--------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|           DISTANCIA|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+--------------------+
|   18:00|         68712|                     33040|          546T-10|        -23.6541035|-46.710938999999996|                 0.0|
|   18:05|         68712|                     33040|          546T-10|       -23.65393525|-46.707456500000006|  0.3551956621027376|
|   18:10|         68712|                     33040|          546T-10|        -23.6543535|      -46.701634625|  0.5947951106073452|
|   18:15|         68712|                     33040|          546T-10|       -23.65908975|        -46.7012395|   0.528182349704729|
|   18:20|         68712|                     33040|          546T-10|      

In [10]:
soma_distancias = dataframe_filter.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')

Soma das distâncias: 11.84 km


In [11]:
rota_linha

[['546T-10', 'Vl. Guacuri - Term. Sto. Amaro', '0082BA']]

In [12]:
id_tracado_linha = cmd.obter_id_tracado_linha(spark_session=spark_session, rota_id=rota_linha[0][0])
id_tracado_linha

[80054, 80055]

In [13]:
tracado_linha = cmd.obter_tracado_linha(spark_session=spark_session,shape_id=id_tracado_linha)
tracado_linha

[[-23.70367431640625, -46.62740707397461],
 [-23.703601837158203, -46.6275520324707],
 [-23.703487396240234, -46.62775421142578],
 [-23.703420639038086, -46.62786102294922],
 [-23.703350067138672, -46.627952575683594],
 [-23.703208923339844, -46.62811279296875],
 [-23.70313835144043, -46.62814712524414],
 [-23.70307731628418, -46.6281852722168],
 [-23.70297622680664, -46.62822341918945],
 [-23.702844619750977, -46.62828063964844],
 [-23.702735900878906, -46.62838363647461],
 [-23.702913284301758, -46.62864685058594],
 [-23.703115463256836, -46.62895584106445],
 [-23.703189849853516, -46.62913513183594],
 [-23.703142166137695, -46.62924575805664],
 [-23.703092575073242, -46.6294059753418],
 [-23.703079223632812, -46.62953567504883],
 [-23.70307159423828, -46.629730224609375],
 [-23.70306396484375, -46.62995147705078],
 [-23.703014373779297, -46.63053894042969],
 [-23.703012466430664, -46.63084030151367],
 [-23.703018188476562, -46.630916595458984],
 [-23.703046798706055, -46.63113403320

In [14]:
posicao_onibus = cmd.obter_posicao_onibus(df_posicao=dataframe_filter)
posicao_onibus

[[-23.6541035, -46.710938999999996],
 [-23.65393525, -46.707456500000006],
 [-23.6543535, -46.701634625],
 [-23.65908975, -46.7012395],
 [-23.6613215, -46.693996],
 [-23.670195, -46.688707],
 [-23.677691000000003, -46.685349],
 [-23.6816385, -46.682483000000005],
 [-23.689941, -46.6788145],
 [-23.695130499999998, -46.655341],
 [-23.695213000000003, -46.6539025],
 [-23.6997765, -46.645726499999995],
 [-23.701917, -46.639164],
 [-23.703032, -46.631026000000006],
 [-23.7031815, -46.625156000000004],
 [-23.704041500000002, -46.6264175],
 [-23.704088499999997, -46.6266995],
 [-23.704049, -46.626543],
 [-23.7036385, -46.627480000000006]]

In [15]:
mapa = cmd.gerar_mapa(posicao_onibus=posicao_onibus, tracado_linha=tracado_linha, cor_linha=rota_linha[0])
mapa